# Weather forecast and actual weather

Using the API, we try to compare the weather forecast of last week with the actual weather occuring on the specific date.
Therefore we use the OpenWeather-API to retreive all necessary information.

In [61]:
%pip install kafka-python
%run Functions.ipynb import *

     |████████████████████████████████| 246 kB 19.7 MB/s 
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [67]:
import requests
import datetime
import logging as log
from kafka import KafkaProducer
from kafka.errors import KafkaError

API_KEY = 'fac9ad607136e9c0f991980b5cd58ed7'

In [46]:
url = 'https://api.openweathermap.org/data/2.5/onecall'
response = requests.get(url, params={
    'lat': 49.4875,
    'lon': 8.4660,
    'exclude': 'current,minutely,hourly,alerts',
    'appid': API_KEY
})

response.json()

{'lat': 49.4875,
 'lon': 8.466,
 'timezone': 'Europe/Berlin',
 'timezone_offset': 7200,
 'daily': [{'dt': 1635591600,
   'sunrise': 1635574274,
   'sunset': 1635610090,
   'moonrise': 1635547680,
   'moonset': 1635603660,
   'moon_phase': 0.8,
   'temp': {'day': 281.95,
    'min': 281.54,
    'max': 283.58,
    'night': 283.58,
    'eve': 282.81,
    'morn': 281.93},
   'feels_like': {'day': 281.95,
    'night': 282.9,
    'eve': 281.18,
    'morn': 281.93},
   'pressure': 1007,
   'humidity': 85,
   'dew_point': 279.57,
   'wind_speed': 3.38,
   'wind_deg': 172,
   'wind_gust': 10.53,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': 80,
   'pop': 0.85,
   'rain': 0.38,
   'uvi': 0.52},
  {'dt': 1635678000,
   'sunrise': 1635660773,
   'sunset': 1635696388,
   'moonrise': 1635638580,
   'moonset': 1635691320,
   'moon_phase': 0.84,
   'temp': {'day': 288.74,
    'min': 283.5,
    'max': 290.89,
    'night': 285.51,
   

In [60]:
json = response.json()
daily_forecasts = json['daily']

forecasts = {
    day: {
        'date': datetime.datetime.fromtimestamp(daily_forecasts[index]['dt']),
        'temp': round(kelvin_to_celsius(daily_forecasts[index]['temp']['day']), 2),
        'rain': daily_forecasts[index]['rain'] if 'rain' in daily_forecasts[index] else 0,
        'snow': daily_forecasts[index]['snow'] if 'snow' in daily_forecasts[index] else 0,
        'clouds': daily_forecasts[index]['clouds'],
        'wind_speed': daily_forecasts[index]['wind_speed']
    }
    for day, index in [('today', 0), ('7 days', -1)]
}

forecasts

{'today': {'date': datetime.datetime(2021, 10, 30, 11, 0),
  'temp': 8.8,
  'rain': 0.38,
  'snow': 0,
  'clouds': 80,
  'wind_speed': 3.38},
 '7 days': {'date': datetime.datetime(2021, 11, 6, 11, 0),
  'temp': 10.32,
  'rain': 0,
  'snow': 0,
  'clouds': 5,
  'wind_speed': 1.9}}

In [68]:
producer = KafkaProducer(bootstrap_servers='localhost:0000')
try:
    producer.send('bda', forecasts)
except KafkaError:
    log.exception('Send error')
    pass

NoBrokersAvailable: NoBrokersAvailable

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa73dd04-f36c-4056-b6c3-896407612102' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>